# Nuclio - Generator function

## Environment

In [6]:
# nuclio: ignore
import nuclio

In [23]:
#%run set_env.ipynb

### Configurations

In [14]:
import os
import datetime

In [15]:
### NOTE
## Platform version dependent

In [18]:
# nuclio: ignore
spec = nuclio.ConfigSpec(config={"spec.triggers.inference.kind":"cron",
                                "spec.triggers.inference.attributes.interval" :"10m",
                                "spec.readinessTimeoutSeconds" : 60,
                                "spec.minReplicas" : 1 },
                         env={'V3IO_HOME' : os.getenv('V3IO_HOME'),
                              'V3IO_ACCESS_KEY' : os.getenv('V3IO_ACCESS_KEY'),
                              'V3IO_USERNAME' : os.getenv('V3IO_USERNAME'),
                              'V3IO_HOME_URL' : os.getenv('V3IO_HOME_URL'),
                              'MONITOR_CONTAINER' : 'bigdata',
                              'MONITOR_TABLE' : 'kubeflow_runs',
                              
                         },
                         cmd=['pip install mlrun v3io'],
                        ).with_v3io().add_volume('/User',os.path.join('/users',os.getenv('V3IO_USERNAME')),kind='v3io',name='v3io')

In [55]:
import v3io.dataplane
import kfp
import pandas as pd
import json

In [41]:
Client = kfp.Client(host='http://ml-pipeline.default-tenant.svc:8888')
v3io_client = v3io.dataplane.Client(context.logger, max_connections=1)

In [77]:
def handler(context,event):
    run_id = event.fields
    if 'run_id' in run_id:
        run_id = str(run_id['run_id'])
        try:
            run_out=Client.get_run(run_id)
            run_status=run_out.to_dict()['run']['status']
        except:
            run_status="Not Found: %s"% run_id
        return run_status
    
    run_items = json.loads(v3io_client.get_items(os.getenv('MONITOR_CONTAINER','bigdata'),os.getenv('MONITOR_TABLE','kubeflow_runs/')).body.decode('utf-8'))
    
    return run_items

## Test

In [78]:
# nuclio: ignore
#event = nuclio.Event(body='',fields={'run_id':'e121acc0-ab1a-41ec-9e0e-1df6bfcbecbe'})
event = nuclio.Event(body='')
output = handler(context, event)
output

{'LastItemIncluded': 'FALSE',
 'NextMarker': 'qwAAAAAAAAAABAAAAAAAAAAAAAAAAAAA',
 'NumItems': 1,
 'Items': [{'__name': {'S': 'dd306c73-f833-4a55-ae68-ffcf43e8f744'},
   'status': {'S': 'started'},
   'id': {'S': 'dd306c73-f833-4a55-ae68-ffcf43e8f744'}}]}

# Deploy

In [37]:
# nuclio: ignore
#%nuclio deploy -p netops -n generator -c
addr = nuclio.deploy_file(name='kfpmonitor',project='monitoring',spec=spec)


[nuclio] 2020-04-20 20:45:44,960 (info) Build complete
[nuclio] 2020-04-20 20:45:53,038 (info) Function deploy complete
[nuclio] 2020-04-20 20:45:53,044 done updating kfpmonitor, function address: 3.12.248.124:32420
